In [1]:
!pip install tslearn
import numpy as np
from keras.optimizers import Adagrad
import matplotlib.pyplot as plt
import pandas as pd

from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import ShapeletModel, grabocka_params_to_shapelet_size_dict

from sklearn.model_selection import cross_validate

    100% |████████████████████████████████| 593kB 16.5MB/s ta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/c6/7d/46/0428fa5dc53f09a5f776ea1fef7a5c5abd79ce07393e9ef52d
Successfully built tslearn


Using TensorFlow backend.


In [2]:
!ls "../input"

PLAsTiCC-2018  features-set


In [3]:
nb_of_passband = 6
dataset_folder = "../input/PLAsTiCC-2018"
!ls {dataset_folder}

data_note.pdf	       test_set_metadata.csv  training_set_metadata.csv
sample_submission.csv  test_set_sample.csv
test_set.csv	       training_set.csv


In [4]:
train = pd.read_csv(f"{dataset_folder}/training_set.csv")
train.head(5)

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [5]:
# train["unix"] = (train.mjd - 40587)*86400
# train.head()

In [6]:
metadata = pd.read_csv(f"{dataset_folder}/training_set_metadata.csv")
metadata.head(5)

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [7]:
train.mjd = train.mjd.astype(np.int32)
train.mjd = train.mjd - train.mjd.min()
train.head(10)

,object_id,mjd,passband,flux,flux_err,detected
0,615,170,2,-544.810303,3.622952,1
1,615,170,1,-816.434326,5.553370,1
2,615,170,3,-471.385529,3.801213,1
3,615,170,4,-388.984985,11.395031,1
4,615,172,2,-681.858887,4.041204,1
5,615,172,1,-1061.457031,6.472994,1
6,615,172,3,-524.954590,3.552751,1
7,615,172,4,-393.480225,3.599346,1
8,615,172,5,-355.886780,10.421921,1
9,615,187,2,-548.013550,3.462291,1


In [8]:
tmp = train.groupby(["object_id", "mjd", "passband"]).count()
tmp[tmp.flux>1].shape[0] / train.shape[0]

0.0013286863308492269

In [9]:
train = train.groupby(["object_id", "mjd", "passband"]).mean().reset_index()
train

,object_id,mjd,passband,flux,flux_err,detected
0,615,170,1,-816.434326,5.553370,1.0
1,615,170,2,-544.810303,3.622952,1.0
2,615,170,3,-471.385529,3.801213,1.0
3,615,170,4,-388.984985,11.395031,1.0
4,615,172,1,-1061.457031,6.472994,1.0
5,615,172,2,-681.858887,4.041204,1.0
6,615,172,3,-524.954590,3.552751,1.0
7,615,172,4,-393.480225,3.599346,1.0
8,615,172,5,-355.886780,10.421921,1.0
9,615,187,1,-815.188599,5.293019,1.0


In [10]:
tmp = train.groupby(["object_id", "mjd", "passband"]).count()
tmp[tmp.flux>1].shape[0] / train.shape[0]

0.0

In [11]:
max_mjd = train.mjd.max()
max_mjd

1094

In [12]:
times = train.groupby(["object_id", "passband"]).apply(lambda row: row.mjd.values).reset_index().rename(columns={0: 'times'})
times.head(5)

,object_id,passband,times
0,615,0,"[239, 240, 241, 242, 243, 271, 294, 295, 296, ..."
1,615,1,"[170, 172, 187, 190, 199, 202, 217, 220, 227, ..."
2,615,2,"[170, 172, 187, 190, 199, 202, 217, 220, 227, ..."
3,615,3,"[170, 172, 187, 190, 199, 202, 217, 220, 227, ..."
4,615,4,"[170, 172, 187, 190, 199, 202, 217, 220, 227, ..."


In [13]:
features_set = pd.read_csv("../input/features-set/feature_set.csv", index_col=[0, 1])
features_set.head(10)

amplitude        ...         weighted_average
    Unnamed: 1                    ...                         
615 0           121.048016        ...               -17.061118
    1           880.533203        ...              -212.397193
    2           646.921722        ...              -102.220639
    3           488.190826        ...              -101.206639
    4           402.069122        ...               -54.744845
    5           400.501618        ...               -59.688379
713 0            14.622504        ...                -3.500958
    1            10.422385        ...                -1.322397
    2            10.298480        ...                -1.030469
    3            11.862454        ...                -1.382941

[10 rows x 8 columns]

In [14]:
def fill_missing_values(times, features_set, max_times=max_mjd, step=1):
    timeset = set(np.arange(0, max_times+1, step))
    X = []
    for i, row in times.iterrows():
        missing = timeset - set(row.times)
        features = features_set.loc[row.object_id, row.passband]
        for t in missing:
            X.append([row.object_id, t, row.passband, features["mean"], features["avg_err"], -1])
    return pd.DataFrame(columns=["object_id", "mjd", "passband", "flux", "flux_err", "detected"], data=X)

In [15]:
missing_data = fill_missing_values(times, features_set, max_mjd)

In [16]:
missing_data.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,0,0,-3.254554,3.823448,-1
1,615,1,0,-3.254554,3.823448,-1
2,615,2,0,-3.254554,3.823448,-1
3,615,3,0,-3.254554,3.823448,-1
4,615,4,0,-3.254554,3.823448,-1


In [17]:
X_train = train.append(missing_data, ignore_index=True)

In [18]:
X_train.to_csv("completed_light_curves.csv")

In [19]:
X_train.sort_values(by="mjd", inplace=True)

In [21]:
train[train.object_id==1920][train.passband==0][train.mjd==471]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,object_id,mjd,passband,flux,flux_err,detected
2921,1920,471,0,3.684292,1.772987,0.0


In [22]:
def format_time_series(X_train):
    X = []
    X_err = []
    for i, row in metadata[["object_id", "target"]].iterrows():
        print(row.object_id)
        ds = X_train[X_train.object_id==row.object_id]
        x, x_err = [], []
        for p in range(nb_of_passband):
            x = np.append(x, ds[ds.passband==p].flux.values)
            x_err = np.append(x_err, ds[ds.passband==p].flux_err.values)
        X.append(x.reshape(max_mjd+1, nb_of_passband))
        X_err.append(x.reshape(max_mjd+1, nb_of_passband))
    X = np.array(X)
    X_err = np.array(X_err)
    return X, X_err

In [23]:
X_ts, X_ts_err = format_time_series(X_train)
y = metadata.target.values
print(X_ts.shape)
print(X_ts_err.shape)
print(y.shape)

615
713
730
745
1124
1227
1598
1632
1920
1926
2072
2103
2300
2330
2624
2677
2922
3041
3285
3423
3489
3910
4088
4132
4171
4173
4220
4389
4595
4819
5527
6180
6266
6762
6947
7033
7164
7315
7409
7566
7698
7703
7756
8328
8688
8745
8784
9006
9172
9184
9203
9543
9936
9985
10321
10337
10349
10478
10586
10757
10796
10798
11165
11359
11507
11770
11773
11931
11978
12695
12872
13079
13138
13194
13459
13482
13504
14080
14156
14279
14398
14539
14553
14601
14674
14983
15002
15251
15475
15626
15674
15700
15718
15845
15968
16339
16349
16463
16496
16802
16983
17094
17172
17285
17366
17370
17515
18029
18507
18556
18645
18706
18937
18952
19154
19213
19866
20567
20934
21335
22184
22574
22901
23116
23127
23299
23373
23396
23409
23539
23795
23822
23848
23857
23931
24193
24236
24592
24849
24903
24947
24989
25003
25039
25474
25529
25577
25783
25920
25925
26161
26338
26352
26401
26531
26660
26783
27124
27339
27941
28220
28301
28391
28636
28843
28915
29088
29252
29416
29420
29576
29668
29670
30066
30172
30191
30

208357
208526
208610
208704
208803
209120
209171
209630
209796
209911
209987
210296
210335
210765
210912
211092
211096
211114
211120
211202
211290
211326
211331
211382
211677
211842
212024
212064
212084
212815
212931
213005
213128
213157
213161
213294
213374
213496
213590
213698
213707
213759
213773
213934
214548
214668
214935
215078
215085
215159
215282
215339
215351
215370
215463
215930
216087
216102
216122
216227
216337
216799
216970
217389
217593
218368
218492
218627
218714
219233
219744
219820
219917
220000
220157
220227
220426
220685
220702
220890
220893
221071
221663
221704
221784
222059
222095
222108
222326
222506
222577
222719
222970
223165
223166
223581
223791
223905
223915
223940
224004
224009
224031
224100
224103
224205
224297
224527
224596
225416
225461
225494
225529
225773
225829
225919
226040
226172
226550
226704
226847
226850
226855
226891
226980
227067
227068
227130
227306
228011
228629
228766
229471
229669
229781
229855
230026
230027
230227
230228
230248
230447
230610

7186711
7186964
7189244
7190427
7209890
7224451
7233276
7253358
7284232
7285136
7289488
7300190
7310089
7312772
7315153
7342033
7342827
7353508
7358574
7364868
7381638
7390884
7412510
7464048
7483838
7484291
7497540
7512735
7512775
7532482
7536832
7544056
7550910
7560475
7571956
7602456
7603574
7609754
7627056
7689256
7700641
7711285
7713926
7779381
7785536
7805020
7808284
7827268
7882307
7911816
7925528
7930361
7930383
7956824
7994928
8056979
8079158
8096230
8146628
8186606
8198259
8247543
8257748
8271656
8298182
8304144
8351755
8366012
8383041
8427624
8466724
8474052
8476473
8490336
8561462
8568272
8577245
8602702
8614569
8626367
8629629
8664713
8667146
8684410
8687161
8698141
8706513
8727043
8740941
8764806
8769352
8773751
8803729
8834794
8843338
8847522
8848545
8917917
8922835
8998750
9033855
9037865
9058890
9079753
9112192
9183851
9210589
9215549
9225328
9240224
9248706
9263357
9310078
9316740
9337823
9373678
9383516
9467509
9492350
9496023
9547880
9569119
9621829
9635060
9679645


27423771
27431216
27445310
27489433
27496433
27519637
27533034
27570837
27613436
27635345
27655935
27698813
27723073
27753416
27760539
27807303
27813710
27840303
27865279
27872720
27883625
27893568
27924241
28006884
28007244
28016462
28065067
28073345
28092580
28132311
28208607
28210173
28233509
28238969
28258880
28282774
28305390
28318934
28334351
28381358
28396939
28413067
28492953
28495414
28499486
28532408
28571862
28596342
28616032
28661491
28667400
28710479
28730623
28735008
28823679
28853121
28883392
28907681
28919347
28920225
28928626
28931793
28978765
29000971
29011171
29042961
29074185
29101600
29102571
29115554
29227269
29241258
29252985
29268274
29283382
29308179
29308415
29330683
29331377
29356260
29361009
29363058
29390759
29399206
29416728
29422697
29465049
29487870
29518677
29557097
29559631
29580219
29586650
29587925
29599343
29609624
29626182
29629882
29654243
29675417
29676720
29725628
29743936
29774433
29824824
29838870
29865365
29902908
29925536
29933389
30059671
3

48567773
48568245
48572207
48586579
48613320
48625068
48670802
48677406
48679089
48704566
48725835
48734543
48741597
48761028
48783156
48837252
48837773
48839513
48850863
48877458
48889152
48951815
49073065
49075963
49101755
49195651
49207610
49256381
49308793
49311602
49328817
49358447
49373192
49374525
49388644
49426863
49428124
49519074
49536340
49551838
49575267
49589687
49611669
49620169
49626003
49679459
49686525
49748338
49766404
49776390
49783859
49789340
49816903
49850869
49875529
49896733
49950170
49983452
50025191
50031767
50036828
50049510
50061624
50088147
50089586
50107953
50118260
50120717
50120780
50136410
50150449
50154052
50198556
50217109
50218120
50228511
50233326
50253232
50382323
50384532
50420178
50423372
50429346
50445945
50483812
50538474
50571533
50669045
50700718
50774437
50787617
50875482
50926314
50935885
50941150
50949719
50952355
50957767
50979853
50996359
51008084
51053939
51055844
51091661
51106790
51137137
51139333
51139578
51160141
51220364
51232774
5

70025523
70048711
70055532
70061525
70078582
70136369
70143431
70155892
70185284
70189146
70215883
70218390
70270634
70317034
70351780
70425954
70505278
70533944
70541274
70576635
70603566
70611740
70649473
70654044
70665843
70672645
70702402
70718336
70757696
70764218
70779589
70799594
70819205
70854573
70863965
70870535
70890576
70906832
70947093
70972197
70987540
70988624
71035086
71037110
71066710
71084419
71098197
71104805
71117832
71126328
71132525
71154324
71171595
71188760
71203310
71274152
71290384
71299170
71306733
71323295
71328417
71399090
71421087
71449444
71451908
71459665
71496808
71498812
71503431
71521763
71565997
71620283
71627745
71694096
71717228
71740669
71765133
71816360
71819477
71829905
71843921
71856115
71878650
71928039
71996356
72009300
72040403
72049251
72058012
72065768
72074395
72074915
72088856
72095866
72101996
72123526
72147416
72147688
72162556
72208309
72221625
72242636
72255258
72262779
72273047
72283619
72341756
72361403
72364882
72391044
72568232
7

90284720
90314147
90318403
90321306
90325864
90361022
90390292
90429960
90432402
90468382
90509277
90512781
90535444
90539476
90562518
90575107
90582474
90614124
90648418
90663647
90700440
90739539
90750519
90768535
90778992
90795265
90799016
90813331
90857932
90880969
90896992
90950273
90976719
90986412
90987882
91022592
91042581
91074470
91091065
91177254
91226665
91252100
91262568
91270103
91293987
91327966
91343957
91367522
91420500
91449873
91470841
91481683
91484679
91569173
91581101
91610803
91625940
91642065
91657831
91676719
91691453
91705234
91732512
91756633
91759166
91760741
91767660
91772215
91836004
91836804
91861189
91900996
91930831
91932161
91963316
91994218
92040236
92045511
92053866
92113420
92114915
92115051
92148656
92243968
92251672
92253169
92254052
92263953
92295640
92296606
92321294
92330883
92332602
92347595
92364022
92401308
92412349
92435141
92436396
92529037
92564906
92577210
92602382
92674135
92728380
92758794
92769326
92769684
92798777
92801741
92819190
9

110034581
110086563
110095987
110099058
110107673
110125812
110134728
110175392
110203727
110205532
110229320
110239521
110245068
110255355
110255618
110261089
110271758
110285813
110317479
110339223
110351591
110354955
110375526
110418764
110439735
110443431
110444195
110448747
110468353
110548280
110567709
110570640
110574404
110609484
110682485
110715196
110720057
110721898
110738539
110750341
110769373
110803113
110804451
110808232
110850947
110852184
110912645
110923057
110925766
110959047
110975108
111077955
111079035
111158769
111171589
111216091
111245629
111271820
111273243
111277141
111294811
111327134
111353970
111366769
111401707
111446272
111467909
111481946
111483418
111494318
111518007
111575039
111575197
111594124
111594831
111601371
111633916
111660438
111679787
111702099
111716677
111722655
111771139
111834074
111941650
111961956
111983701
111997875
112007948
112031229
112049038
112060173
112061925
112097340
112099006
112103202
112105084
112109284
112131416
112147113


128664786
128678785
128698690
128699652
128735416
128761918
128765670
128796733
128814583
128839699
128895075
128900162
128926736
128932972
128949060
128953634
128961480
128966389
128971260
128983448
128989049
128992591
129003634
129019585
129023853
129030512
129041496
129045287
129049932
129073978
129135543
129175387
129205942
129217551
129237530
129257128
129259953
129277104
129296519
129311413
129328649
129329222
129335528
129336792
129391307
129399030
129416892
129459169
129463916
129524163
129546183
129554871
129640199
129687910
129705825
129722950
129729789
129759341
129773611
129818444
129828542
129832862
129845065
129913235
129937154
129964738
129999603
130062639
130072052
130085491
130088373
130102264
130102752
130108924
130127657
130188020
130219752
130231675
130263372
130312781
130319749
130330088
130359176
130375489
130386135
130402542
130408188
130414189
130489916
130552230
130595291
130617044
130622528
130639669
130659834
130678775
130684460
130695262
130698059
130716752


In [ ]:
X_ts /= (X_ts_err + 1) # +1 to avoid zero division

In [24]:
X_ts = np.nan_to_num(X_ts)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


In [25]:
X_ts = TimeSeriesScalerMinMax().fit_transform(X_ts)
X_ts

/opt/conda/lib/python3.6/site-packages/tslearn/preprocessing.py:104: RuntimeWarning: invalid value encountered in subtract
  X_ = (X_ - min_t) * (self.max_ - self.min_) / range_t + self.min_
/opt/conda/lib/python3.6/site-packages/tslearn/preprocessing.py:104: RuntimeWarning: invalid value encountered in true_divide
  X_ = (X_ - min_t) * (self.max_ - self.min_) / range_t + self.min_


array([[[1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        ],
        [1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        ],
        [1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        ],
        ...,
        [0.26255672, 0.13936003, 0.39488098, 0.1691817 , 0.22270791,
         0.31970868],
        [0.26255672, 0.13936003, 0.39488098, 0.1691817 , 0.22270791,
         0.31970868],
        [0.26255672, 0.13936003, 0.39488098, 0.1691817 , 0.22270791,
         0.31970868]],

       [[0.60766135, 0.60766135, 0.60766135, 0.60766135, 0.60766135,
         0.60766135],
        [0.60766135, 0.60766135, 0.60766135, 0.60766135, 0.60766135,
         0.60766135],
        [0.60766135, 0.60766135, 0.60766135, 0.60766135, 0.60766135,
         0.60766135],
        ...,
        [0.61299006, 0.61299006, 0.61299006, 0.61299006, 0.61299006,
         0.60487852],
        [0.61299006, 0.61299006, 0.60370996, 0.61299006, 0.612

In [26]:
y

array([92, 88, 42, ..., 16, 65,  6])

In [27]:
shapelet_sizes = grabocka_params_to_shapelet_size_dict(n_ts=X_ts.shape[0],
                                                       ts_sz=X_ts.shape[1],
                                                       n_classes=len(set(y)),
                                                       l=0.1,
                                                       r=2)
shapelet_sizes

{109: 8, 218: 7}

In [44]:
shp_clf = ShapeletModel(n_shapelets_per_size=shapelet_sizes,
                        optimizer=Adagrad(lr=.1),
                        weight_regularizer=.01,
                        max_iter=50,
                        verbose_level=0)

In [31]:
def get_best(model, X, y):
    cv_result = cross_validate(model, X, y, cv=4,return_estimator=True, scoring="neg_log_loss")
    best_index = np.argmax(cv_result["test_score"])
    print(f"Log Error: {-cv_result['test_score'][best_index]}")
    return cv_result["estimator"][best_index]

In [ ]:
shp_clf.fit(X_ts, y)

In [43]:
shp_clf = get_best(shp_clf, X_ts, y)

TypeError: can't pickle _thread.RLock objects